# 
# JSON structured data from tables in scientific papers

## Jsonformer non-token approach for OpenAI
#### (based on: https://github.com/1rgs/jsonformer, https://github.com/martinezpl/jsonformer/tree/add-openai)

In [1]:
import os
import pandas as pd
import json
import time

import openai
from IPython.display import IFrame
from IPython.display import HTML

from LLMsTablesToJson.html_table import get_driver, extract_tableSource, extract_table, quit_driver
from LLMsTablesToJson.jsonformer_non_tokens import JsonformerNoTokens, OpenAIModel, highlight_values

DATA_DIR = "your_path"
chromedriver_path = os.path.join(DATA_DIR, "chromedriver.exe")

openai.api_key = "your_openai_api_key"

## Example: Catalysts for CO<sub>2</sub>  Fischer–Tropsch conversion to liquid fuels

### Get table HTML code from DOI

In [39]:
doi = "10.1021/acs.energyfuels.1c02406"

In [40]:
IFrame(src="https://dx.doi.org/" + doi, width=900, height=380)

In [41]:
driver = get_driver(doi, chromedriver_path)
time.sleep(2)
tableSource = extract_tableSource(driver)

[<selenium.webdriver.remote.webelement.WebElement (session="0660c5e899aeb84c6af871e4c8f3c539", element="C02A430DE0C72BFE36D96BA27F037805_element_79")>, <selenium.webdriver.remote.webelement.WebElement (session="0660c5e899aeb84c6af871e4c8f3c539", element="C02A430DE0C72BFE36D96BA27F037805_element_80")>, <selenium.webdriver.remote.webelement.WebElement (session="0660c5e899aeb84c6af871e4c8f3c539", element="C02A430DE0C72BFE36D96BA27F037805_element_81")>, <selenium.webdriver.remote.webelement.WebElement (session="0660c5e899aeb84c6af871e4c8f3c539", element="C02A430DE0C72BFE36D96BA27F037805_element_82")>, <selenium.webdriver.remote.webelement.WebElement (session="0660c5e899aeb84c6af871e4c8f3c539", element="C02A430DE0C72BFE36D96BA27F037805_element_83")>, <selenium.webdriver.remote.webelement.WebElement (session="0660c5e899aeb84c6af871e4c8f3c539", element="C02A430DE0C72BFE36D96BA27F037805_element_84")>, <selenium.webdriver.remote.webelement.WebElement (session="0660c5e899aeb84c6af871e4c8f3c539",

In [42]:
table_num = 6
html_table_catalystCO2 = extract_table(tableSource, table_num)
quit_driver(driver)

In [43]:
html_table_catalystCO2

'<table class="table "><colgroup><col align="left"><col align="left"><col align="left"><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="left"></colgroup><thead><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">catalyst</th><th class="colsep0 rowsep0" scope="col" align="center">synthesis method</th><th class="colsep0 rowsep0" scope="col" align="center">promoter incorporation</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>X</i>&nbsp;CO<sub>2</sub> (%)</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>S</i>&nbsp;CO (%)</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>S</i>&nbsp;C<sub>1</sub> (%)</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>S</i>&nbsp;C<sub>2</sub>–C<sub>4</sub> (%)</th><th class="colsep0 rowsep

In [44]:
with open("html_table_catalystCO2.txt", "w") as file:
    file.write(html_table_catalystCO2)

### Show table

In [46]:
HTML(html_table_catalystCO2)

catalyst,synthesis method,promoter incorporation,X CO2 (%),S CO (%),S C1 (%),S C2–C4 (%),S C5+ (%),temperature (°C),P (MPa),reference
Fe–Cu–K,triple incipient wetness impregnation,impregnation,15,50,11.00,21.00,18.00,350,8.6,(64)
Fe–PYL,hydrothermal,hydrothermal,21.6,29.2,36.80,46.00,17.20,300,1,(65)
Fe–K,templated synthesis,ultrasonic-assisted melt infiltration,50.6,8.2,15.42,31.95,44.52,300,2.5,(66)
MnFe–N2,dry impregnation,impregnation,25.2,35.6,39.8,21.92,1.00,360,2.5,(67)
Fe2O3,precipitation,cellulose templated,40,15,11.9,36.98,36.04,300,1.5,(68)


### Convert HTML table to JSON format with openAI LLMs

In [54]:
json_schema_catalystCO2 = {
    "type": "object",
    "properties": {
        "Carbon-supported catalysts CO2": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "Catalyst": {"type": "string"},
                    "Synthesis method": {"type": "string"},
                    "Promoter incorporation": {"type": "string"},
                    "X CO2 (%)": {"type": "number"},
                    "S CO (%)": {"type": "number"},
                    "S C1 (%)": {"type": "number"},
                    "S C2–C4 (%)": {"type": "number"},
                    "S C5+ (%)": {"type": "number"},
                    "Temperature (°C)": {"type": "number"},
                    "P (MPa)": {"type": "number"},
                },
            },
        },
    },
}

In [55]:
with open('json_schema_catalystCO2.json', 'w') as file:
    json.dump(json_schema_catalystCO2, file)
with open("json_schema_catalystCO2.json") as file:
    json_schema_catalystCO2 = json.load(file)
    file.close()

In [56]:
json_schema_catalystCO2

{'type': 'object',
 'properties': {'Carbon-supported catalysts CO2': {'type': 'array',
   'items': {'type': 'object',
    'properties': {'Catalyst': {'type': 'string'},
     'Synthesis method': {'type': 'string'},
     'Promoter incorporation': {'type': 'string'},
     'X CO2 (%)': {'type': 'number'},
     'S CO (%)': {'type': 'number'},
     'S C1 (%)': {'type': 'number'},
     'S C2–C4 (%)': {'type': 'number'},
     'S C5+ (%)': {'type': 'number'},
     'Temperature (°C)': {'type': 'number'},
     'P (MPa)': {'type': 'number'}}}}}}

In [57]:
builder = JsonformerNoTokens(
    model=OpenAIModel("text-davinci-003", debug=False),
    json_schema=json_schema_catalystCO2,
    text=html_table_catalystCO2,
    prompt="Generate an object with the following schema extracting the information from the provided table in html code:",
    temperature=0.5,
    debug=True,
    max_array_length=5,
    max_string_token_length=12,
)

print("Generating...")
result_catalystCO2 = builder()

Generating...
[generate_string] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table class="table "><colgroup><col align="left"><col align="left"><col align="left"><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="left"></colgroup><thead><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">catalyst</th><th class="colsep0 rowsep0" scope="col" align="center">synthesis method</th><th class="colsep0 rowsep0" scope="col" align="center">promoter incorporation</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>X</i>&nbsp;CO<sub>2</sub> (%)</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>S</i>&nbsp;CO (%)</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>S</i>&nbsp;C<sub>1</sub> 

[generate_string] response  "triple incipient wetness impregnation", "
[generate_string] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table class="table "><colgroup><col align="left"><col align="left"><col align="left"><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="left"></colgroup><thead><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">catalyst</th><th class="colsep0 rowsep0" scope="col" align="center">synthesis method</th><th class="colsep0 rowsep0" scope="col" align="center">promoter incorporation</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>X</i>&nbsp;CO<sub>2</sub> (%)</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>S</i>&nbsp;CO (%)</th><th class="colsep0 rowsep0" scope="

[generate_number] response  15, "S CO (%)
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table class="table "><colgroup><col align="left"><col align="left"><col align="left"><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="left"></colgroup><thead><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">catalyst</th><th class="colsep0 rowsep0" scope="col" align="center">synthesis method</th><th class="colsep0 rowsep0" scope="col" align="center">promoter incorporation</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>X</i>&nbsp;CO<sub>2</sub> (%)</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>S</i>&nbsp;CO (%)</th><th class="colsep0 rowsep0" scope="col" align="center" char=".">

[generate_number] response  11.00, "S
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table class="table "><colgroup><col align="left"><col align="left"><col align="left"><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="left"></colgroup><thead><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">catalyst</th><th class="colsep0 rowsep0" scope="col" align="center">synthesis method</th><th class="colsep0 rowsep0" scope="col" align="center">promoter incorporation</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>X</i>&nbsp;CO<sub>2</sub> (%)</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>S</i>&nbsp;CO (%)</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>S

[generate_number] response  18.0, "Temperature
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table class="table "><colgroup><col align="left"><col align="left"><col align="left"><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="left"></colgroup><thead><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">catalyst</th><th class="colsep0 rowsep0" scope="col" align="center">synthesis method</th><th class="colsep0 rowsep0" scope="col" align="center">promoter incorporation</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>X</i>&nbsp;CO<sub>2</sub> (%)</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>S</i>&nbsp;CO (%)</th><th class="colsep0 rowsep0" scope="col" align="center" char

...

[generate_string] response  "precipitation", "Promoter incorporation": "
[generate_string] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table class="table "><colgroup><col align="left"><col align="left"><col align="left"><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="left"></colgroup><thead><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">catalyst</th><th class="colsep0 rowsep0" scope="col" align="center">synthesis method</th><th class="colsep0 rowsep0" scope="col" align="center">promoter incorporation</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>X</i>&nbsp;CO<sub>2</sub> (%)</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>S</i>&nbsp;CO (%)</th><th class="colsep0 rowsep0" scope

[generate_number] response  40, "S CO (%)
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table class="table "><colgroup><col align="left"><col align="left"><col align="left"><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="left"></colgroup><thead><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">catalyst</th><th class="colsep0 rowsep0" scope="col" align="center">synthesis method</th><th class="colsep0 rowsep0" scope="col" align="center">promoter incorporation</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>X</i>&nbsp;CO<sub>2</sub> (%)</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>S</i>&nbsp;CO (%)</th><th class="colsep0 rowsep0" scope="col" align="center" char=".">

[generate_number] response  11.9, "S
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table class="table "><colgroup><col align="left"><col align="left"><col align="left"><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="left"></colgroup><thead><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">catalyst</th><th class="colsep0 rowsep0" scope="col" align="center">synthesis method</th><th class="colsep0 rowsep0" scope="col" align="center">promoter incorporation</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>X</i>&nbsp;CO<sub>2</sub> (%)</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>S</i>&nbsp;CO (%)</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>S<

[generate_number] response  36.04, "Temperature
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table class="table "><colgroup><col align="left"><col align="left"><col align="left"><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="left"></colgroup><thead><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">catalyst</th><th class="colsep0 rowsep0" scope="col" align="center">synthesis method</th><th class="colsep0 rowsep0" scope="col" align="center">promoter incorporation</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>X</i>&nbsp;CO<sub>2</sub> (%)</th><th class="colsep0 rowsep0" scope="col" align="center" char="."><i>S</i>&nbsp;CO (%)</th><th class="colsep0 rowsep0" scope="col" align="center" cha

[generate_number] response  1.5}]}


In [58]:
highlight_values(result_catalystCO2)

{
  Carbon-supported catalysts CO2: [
    {
      Catalyst: "Fe–Cu–K",
      Synthesis method: "triple incipient wetness impregnation",
      Promoter incorporation: "impregnation",
      X CO2 (%): 15,
      S CO (%): 50,
      S C1 (%): 11.0,
      S C2–C4 (%): 21.0,
      S C5+ (%): 18.0,
      Temperature (°C): 350,
      P (MPa): 8.6
    },
    {
      Catalyst: "Fe\u2013PYL",
      Synthesis method: "hydrothermal",
      Promoter incorporation: "hydrothermal",
      X CO2 (%): 21.6,
      S CO (%): 29.2,
      S C1 (%): 36.8,
      S C2–C4 (%): 46.0,
      S C5+ (%): 17.2,
      Temperature (°C): 300,
      P (MPa): 1.0
    },
    {
      Catalyst: "Fe\\u2013K",
      Synthesis method: "templated synthesis",
      Promoter incorporation: "ultrasonic-assisted melt infiltration",
      X CO2 (%): 50.6,
      S CO (%): 8.2,
      S C1 (%): 15.42,
      S C2–C4 (%): 31.95,
      S C5+ (%): 44.52,
      Temperature (°C): 300,
      P (MPa): 2.5
    },
    {
      Catalyst: "MnFe\u2013

In [59]:
with open('json_table_catalystCO2.json', 'w') as file:
    json.dump(result_catalystCO2, file)